In [1]:
import cv2
import mediapipe as mp #face detector
import math
import numpy as np
import time
import datetime
from collections import Counter

import warnings
warnings.simplefilter("ignore", UserWarning)

import torch
import torch.nn as  nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

# Import tkinter for GUI
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog
import PIL.Image, PIL.ImageTk
from threading import Thread, Event
import os

#### Model architectures

In [2]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0.99)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding='same', bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0.99)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0, bias=False)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion, eps=0.001, momentum=0.99)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Conv2dSame(torch.nn.Conv2d):

    def calc_same_pad(self, i: int, k: int, s: int, d: int) -> int:
        return max((math.ceil(i / s) - 1) * s + (k - 1) * d + 1 - i, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ih, iw = x.size()[-2:]

        pad_h = self.calc_same_pad(i=ih, k=self.kernel_size[0], s=self.stride[0], d=self.dilation[0])
        pad_w = self.calc_same_pad(i=iw, k=self.kernel_size[1], s=self.stride[1], d=self.dilation[1])

        if pad_h > 0 or pad_w > 0:
            x = F.pad(
                x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2]
            )
        return F.conv2d(
            x,
            self.weight,
            self.bias,
            self.stride,
            self.padding,
            self.dilation,
            self.groups,
        )

class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv_layer_s2_same = Conv2dSame(num_channels, 64, 7, stride=2, groups=1, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64, eps=0.001, momentum=0.99)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64, stride=1)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(512*ResBlock.expansion, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def extract_features(self, x):
        x = self.relu(self.batch_norm1(self.conv_layer_s2_same(x)))
        x = self.max_pool(x)
        # print(x.shape)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x
        
    def forward(self, x):
        x = self.extract_features(x)
        x = self.relu1(x)
        x = self.fc2(x)
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride, bias=False, padding=0),
                nn.BatchNorm2d(planes*ResBlock.expansion, eps=0.001, momentum=0.99)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)
        
def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)


class LSTMPyTorch(nn.Module):
    def __init__(self):
        super(LSTMPyTorch, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size=512, hidden_size=512, batch_first=True, bidirectional=False)
        self.lstm2 = nn.LSTM(input_size=512, hidden_size=256, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(256, 7)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)        
        x = self.fc(x[:, -1, :])
        x = self.softmax(x)
        return x

#### Sub functions

In [3]:
def pth_processing(fp):
    class PreprocessInput(torch.nn.Module):
        def init(self):
            super(PreprocessInput, self).init()

        def forward(self, x):
            x = x.to(torch.float32)
            x = torch.flip(x, dims=(0,))
            x[0, :, :] -= 91.4953
            x[1, :, :] -= 103.8827
            x[2, :, :] -= 131.0912
            return x

    def get_img_torch(img):
        
        ttransform = transforms.Compose([
            transforms.PILToTensor(),
            PreprocessInput()
        ])
        img = img.resize((224, 224), Image.Resampling.NEAREST)
        img = ttransform(img)
        img = torch.unsqueeze(img, 0)
        return img
    return get_img_torch(fp)

def tf_processing(fp):
    def preprocess_input(x):
        x_temp = np.copy(x)
        x_temp = x_temp[..., ::-1]
        x_temp[..., 0] -= 91.4953
        x_temp[..., 1] -= 103.8827
        x_temp[..., 2] -= 131.0912
        return x_temp

    def get_img_tf(img):
        img = cv2.resize(img, (224,224), interpolation=cv2.INTER_NEAREST)
        img = tf.keras.utils.img_to_array(img)
        img = preprocess_input(img)
        img = np.array([img])
        return img

    return get_img_tf(fp)

def norm_coordinates(normalized_x, normalized_y, image_width, image_height):
    
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    
    return x_px, y_px

def get_box(fl, w, h):
    idx_to_coors = {}
    for idx, landmark in enumerate(fl.landmark):
        landmark_px = norm_coordinates(landmark.x, landmark.y, w, h)

        if landmark_px:
            idx_to_coors[idx] = landmark_px

    x_min = np.min(np.asarray(list(idx_to_coors.values()))[:,0])
    y_min = np.min(np.asarray(list(idx_to_coors.values()))[:,1])
    endX = np.max(np.asarray(list(idx_to_coors.values()))[:,0])
    endY = np.max(np.asarray(list(idx_to_coors.values()))[:,1])

    (startX, startY) = (max(0, x_min), max(0, y_min))
    (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
    
    return startX, startY, endX, endY

#### Emotion Logging GUI Application

In [4]:
class EmotionLoggingApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.window.geometry("1200x700")
        
        # Initialize models
        self.init_models()
        
        # Create the UI elements
        self.create_ui()
        
        # Initialize variables
        self.is_running = False
        self.stop_event = Event()
        self.emotion_logs = []
        self.current_emotions = []
        self.logging_start_time = None
        self.last_log_time = None
        self.transcript_file_path = None
        self.stopwatch_active = False
        self.elapsed_time = 0
        
        # Start video capture
        self.cap = cv2.VideoCapture(0)
        self.update()
        
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.window.mainloop()
    
    def init_models(self):
        # MediaPipe setup
        self.mp_face_mesh = mp.solutions.face_mesh
        
        # Model settings
        name_backbone_model = 'FER_static_ResNet50_AffectNet.pt'
        name_LSTM_model = 'Aff-Wild2'
        
        # Load ResNet model
        self.pth_backbone_model = ResNet50(7, channels=3)
        self.pth_backbone_model.load_state_dict(torch.load(name_backbone_model))
        self.pth_backbone_model.eval()
        
        # Load LSTM model
        self.pth_LSTM_model = LSTMPyTorch()
        self.pth_LSTM_model.load_state_dict(torch.load(f'FER_dinamic_LSTM_{name_LSTM_model}.pt'))
        self.pth_LSTM_model.eval()
        
        # Emotion dictionary
        self.DICT_EMO = {0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger'}
        self.NEGATIVE_EMOTIONS = ['Sadness', 'Fear', 'Disgust', 'Anger']
        
        # Initialize LSTM features
        self.lstm_features = []
    
    def create_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.window)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left frame for video and transcript
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Transcript frame at the top of left frame
        transcript_frame = ttk.LabelFrame(left_frame, text="Presentation Transcript")
        transcript_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Transcript buttons frame
        transcript_buttons_frame = ttk.Frame(transcript_frame)
        transcript_buttons_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Choose file button
        self.choose_file_button = ttk.Button(transcript_buttons_frame, text="Choose Transcript File", command=self.choose_transcript_file)
        self.choose_file_button.pack(side=tk.LEFT, padx=5)
        
        # File name label
        self.file_label = ttk.Label(transcript_buttons_frame, text="No file selected")
        self.file_label.pack(side=tk.LEFT, padx=5)
        
        # Transcript text area
        self.transcript_text = scrolledtext.ScrolledText(transcript_frame, wrap=tk.WORD, height=8)
        self.transcript_text.pack(fill=tk.X, padx=5, pady=5)
        self.transcript_text.insert(tk.END, "Load a transcript file to view your presentation text here.")
        
        # Video frame
        self.video_frame = ttk.LabelFrame(left_frame, text="Webcam Feed")
        self.video_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Create canvas for video
        self.canvas = tk.Canvas(self.video_frame)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Right frame for controls and summary
        right_frame = ttk.Frame(main_frame, width=300)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, padx=5, pady=5, expand=False)
        right_frame.pack_propagate(False)  # Prevent the frame from shrinking to fit its contents
        
        # Control frame
        control_frame = ttk.LabelFrame(right_frame, text="Controls")
        control_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Stopwatch display
        stopwatch_frame = ttk.Frame(control_frame)
        stopwatch_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Label(stopwatch_frame, text="Elapsed Time: ").pack(side=tk.LEFT)
        self.time_label = ttk.Label(stopwatch_frame, text="00:00:00", font=("Arial", 14, "bold"))
        self.time_label.pack(side=tk.LEFT, padx=5)
        
        # Start button
        self.start_button = ttk.Button(control_frame, text="Start Logging", command=self.start_logging)
        self.start_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Stop button
        self.stop_button = ttk.Button(control_frame, text="Stop Logging", command=self.stop_logging, state=tk.DISABLED)
        self.stop_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Reset button
        self.reset_button = ttk.Button(control_frame, text="Reset", command=self.reset_logging, state=tk.DISABLED)
        self.reset_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Status indicator
        status_frame = ttk.Frame(control_frame)
        status_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Label(status_frame, text="Status: ").pack(side=tk.LEFT)
        self.status_label = ttk.Label(status_frame, text="Ready")
        self.status_label.pack(side=tk.LEFT)
        
        # Summary frame
        summary_frame = ttk.LabelFrame(right_frame, text="Emotion Summary")
        summary_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Create scrolled text for summary
        self.summary_text = scrolledtext.ScrolledText(summary_frame, wrap=tk.WORD, width=30, height=20)
        self.summary_text.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        self.summary_text.config(state=tk.DISABLED)
    
    def choose_transcript_file(self):
        file_path = filedialog.askopenfilename(
            title="Select Transcript File",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        
        if file_path:
            self.transcript_file_path = file_path
            filename = os.path.basename(file_path)
            self.file_label.config(text=f"Selected: {filename}")
            
            try:
                with open(file_path, 'r') as file:
                    content = file.read()
                    self.transcript_text.delete(1.0, tk.END)
                    self.transcript_text.insert(tk.END, content)
            except Exception as e:
                self.transcript_text.delete(1.0, tk.END)
                self.transcript_text.insert(tk.END, f"Error reading file: {str(e)}")
    
    def update_stopwatch(self):
        if self.stopwatch_active:
            current_time = time.time()
            self.elapsed_time = current_time - self.logging_start_time
            
            # Format time as HH:MM:SS
            hours, remainder = divmod(int(self.elapsed_time), 3600)
            minutes, seconds = divmod(remainder, 60)
            time_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            self.time_label.config(text=time_str)
            
            # Update every 1 second
            self.window.after(1000, self.update_stopwatch)
    
    def update(self):
        ret, frame = self.cap.read()
        
        if ret:
            # Process frame if logging is active
            if self.is_running:
                self.process_frame(frame)
            
            # Convert to RGB for display
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = PIL.Image.fromarray(frame_rgb)
            
            # Resize to fit canvas
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            if canvas_width > 1 and canvas_height > 1:
                ratio = min(canvas_width/img.width, canvas_height/img.height)
                new_width = int(img.width * ratio)
                new_height = int(img.height * ratio)
                img = img.resize((new_width, new_height), PIL.Image.Resampling.LANCZOS)
            
            self.photo = PIL.ImageTk.PhotoImage(image=img)
            self.canvas.create_image(canvas_width//2, canvas_height//2, image=self.photo, anchor=tk.CENTER)
        
        if not self.stop_event.is_set():
            self.window.after(10, self.update)
    
    def process_frame(self, frame):
        current_time = time.time()
        
        # Initialize the 10-second logging interval
        if self.last_log_time is None:
            self.last_log_time = current_time
        
        # Process frame for emotion detection
        frame_copy = frame.copy()
        frame_copy.flags.writeable = False
        frame_copy = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
        
        with self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:
            
            results = face_mesh.process(frame_copy)
            
            if results.multi_face_landmarks:
                for fl in results.multi_face_landmarks:
                    h, w, _ = frame_copy.shape
                    startX, startY, endX, endY = get_box(fl, w, h)
                    
                    # Extract face
                    cur_face = frame_copy[startY:endY, startX:endX]
                    if cur_face.size == 0:  # Skip if face not properly detected
                        continue
                    
                    try:
                        # Process with model
                        cur_face = pth_processing(Image.fromarray(cur_face))
                        features = torch.nn.functional.relu(self.pth_backbone_model.extract_features(cur_face)).detach().numpy()
                        
                        # Update LSTM features
                        if len(self.lstm_features) == 0:
                            self.lstm_features = [features] * 10
                        else:
                            self.lstm_features = self.lstm_features[1:] + [features]
                        
                        lstm_f = torch.from_numpy(np.vstack(self.lstm_features))
                        lstm_f = torch.unsqueeze(lstm_f, 0)
                        output = self.pth_LSTM_model(lstm_f).detach().numpy()
                        
                        # Get emotion label
                        cl = np.argmax(output)
                        emotion = self.DICT_EMO[cl]
                        confidence = output[0][cl]
                        
                        # Add to current emotions
                        self.current_emotions.append(emotion)
                    except Exception as e:
                        print(f"Error processing face: {e}")
        
        # Log emotions every 10 seconds
        if current_time - self.last_log_time >= 10 and self.current_emotions:
            self.log_emotions()
            self.last_log_time = current_time
    
    def log_emotions(self):
        if not self.current_emotions:
            return
        
        # Count occurrences of each emotion
        emotion_counts = Counter(self.current_emotions)
        dominant_emotion = emotion_counts.most_common(1)[0][0]
        
        # Check for negative emotions
        negative_emotions = [emotion for emotion in self.current_emotions if emotion in self.NEGATIVE_EMOTIONS]
        has_negative = len(negative_emotions) > 0
        
        # Create log entry
        elapsed_time = int(time.time() - self.logging_start_time)
        timestamp = f"{elapsed_time//60:02d}:{elapsed_time%60:02d}"
        
        log_entry = {
            "timestamp": timestamp,
            "dominant_emotion": dominant_emotion,
            "counts": dict(emotion_counts),
            "has_negative": has_negative
        }
        
        self.emotion_logs.append(log_entry)
        
        # Clear current emotions for next interval
        self.current_emotions = []
    
    def start_logging(self):
        self.is_running = True
        self.logging_start_time = time.time()
        self.last_log_time = None
        self.emotion_logs = []
        self.current_emotions = []
        
        # Start stopwatch
        self.stopwatch_active = True
        self.update_stopwatch()
        
        # Update UI
        self.status_label.config(text="Logging")
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.reset_button.config(state=tk.DISABLED)
        self.choose_file_button.config(state=tk.DISABLED)  # Disable file selection during recording
        
        # Clear summary
        self.summary_text.config(state=tk.NORMAL)
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, "Logging emotions...\n")
        self.summary_text.config(state=tk.DISABLED)
    
    def stop_logging(self):
        if self.is_running:
            self.is_running = False
            self.stopwatch_active = False
            
            # Log any remaining emotions
            if self.current_emotions:
                self.log_emotions()
            
            # Update UI
            self.status_label.config(text="Ready")
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.DISABLED)
            self.reset_button.config(state=tk.NORMAL)
            self.choose_file_button.config(state=tk.NORMAL)  # Re-enable file selection
            
            # Generate and display summary
            self.display_summary()
    
    def reset_logging(self):
        # Reset all variables to initial state
        self.emotion_logs = []
        self.current_emotions = []
        self.logging_start_time = None
        self.last_log_time = None
        self.elapsed_time = 0
        self.lstm_features = []
        
        # Reset UI
        self.time_label.config(text="00:00:00")
        self.status_label.config(text="Ready")
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.reset_button.config(state=tk.DISABLED)
        
        # Clear summary
        self.summary_text.config(state=tk.NORMAL)
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, "Ready to start a new session.\n")
        self.summary_text.config(state=tk.DISABLED)
    
    def display_summary(self):
        self.summary_text.config(state=tk.NORMAL)
        self.summary_text.delete(1.0, tk.END)
        
        if not self.emotion_logs:
            self.summary_text.insert(tk.END, "No emotions logged.")
            self.summary_text.config(state=tk.DISABLED)
            return
        
        # Format final time
        hours, remainder = divmod(int(self.elapsed_time), 3600)
        minutes, seconds = divmod(remainder, 60)
        time_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        
        self.summary_text.insert(tk.END, f"== Emotion Logging Summary ==\n")
        self.summary_text.insert(tk.END, f"Total Duration: {time_str}\n")
        
        # Add transcript file info if available
        if self.transcript_file_path:
            filename = os.path.basename(self.transcript_file_path)
            self.summary_text.insert(tk.END, f"Transcript: {filename}\n")
        
        self.summary_text.insert(tk.END, "\n")
        
        # Per 10-second interval summary
        self.summary_text.insert(tk.END, "=== 10-second Intervals ===\n")
        for i, log in enumerate(self.emotion_logs):
            self.summary_text.insert(tk.END, f"[{log['timestamp']}] ")
            self.summary_text.insert(tk.END, f"Dominant: {log['dominant_emotion']}\n")
            
            if log['has_negative']:
                self.summary_text.insert(tk.END, "⚠️ Negative emotions detected!\n")
            
            # Show emotion distribution
            for emotion, count in log['counts'].items():
                percentage = count / sum(log['counts'].values()) * 100
                self.summary_text.insert(tk.END, f"  - {emotion}: {percentage:.1f}%\n")
            
            self.summary_text.insert(tk.END, "\n")
        
        # Overall summary
        self.summary_text.insert(tk.END, "=== Overall Summary ===\n")
        
        # Count dominant emotions across all intervals
        dominant_counts = Counter([log['dominant_emotion'] for log in self.emotion_logs])
        most_common = dominant_counts.most_common()
        
        self.summary_text.insert(tk.END, "Most frequent emotions:\n")
        for emotion, count in most_common:
            percentage = count / len(self.emotion_logs) * 100
            self.summary_text.insert(tk.END, f"  - {emotion}: {percentage:.1f}%\n")
        
        # Check for negative emotions
        negative_intervals = sum(1 for log in self.emotion_logs if log['has_negative'])
        if negative_intervals > 0:
            neg_percentage = negative_intervals / len(self.emotion_logs) * 100
            self.summary_text.insert(tk.END, f"\n⚠️ Negative emotions detected in {negative_intervals} intervals ({neg_percentage:.1f}%)\n")
        else:
            self.summary_text.insert(tk.END, "\n✅ No negative emotions detected\n")
        
        # Provide presentation advice based on emotions
        self.summary_text.insert(tk.END, "\n=== Presentation Advice ===\n")
        
        # Tailor advice based on detected emotions
        if negative_intervals > len(self.emotion_logs) * 0.3:  # If more than 30% negative
            self.summary_text.insert(tk.END, "Your rehearsal showed significant stress or negative emotions. ")
            self.summary_text.insert(tk.END, "Consider practicing more to build confidence. ")
            self.summary_text.insert(tk.END, "Focus on deep breathing before starting.\n")
        elif 'Happiness' in dominant_counts and dominant_counts['Happiness'] > len(self.emotion_logs) * 0.4:
            self.summary_text.insert(tk.END, "Great job! You appear confident and positive during your presentation. ")
            self.summary_text.insert(tk.END, "Keep this energy for your actual presentation.\n")
        elif 'Neutral' in dominant_counts and dominant_counts['Neutral'] > len(self.emotion_logs) * 0.7:
            self.summary_text.insert(tk.END, "Your presentation appears quite neutral. ")
            self.summary_text.insert(tk.END, "Try adding more vocal variety and enthusiasm to engage your audience.\n")
        else:
            self.summary_text.insert(tk.END, "Your presentation showed mixed emotions. ")
            self.summary_text.insert(tk.END, "Focus on maintaining consistent positive energy throughout.\n")
        
        self.summary_text.config(state=tk.DISABLED)
    
    def on_closing(self):
        self.stop_event.set()
        if self.cap and self.cap.isOpened():
            self.cap.release()
        self.window.destroy()

In [5]:
# Run the application
root = tk.Tk()
app = EmotionLoggingApp(root, "Presentation Rehearsal Emotion Tracker")

2025-07-12 12:14:48.040 Python[92057:4725363] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1752347706.083158 4725363 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1752347706.085445 4726067 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752347706.087915 4726066 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1752347706.232072 4725363 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
W0000 00:00:1752347706.233314 4726077 inference_feedback_manager.cc:114] Feedback manager requires a model with a 